In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

In [2]:
import sys

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.other.read_data_and_build_rd import *

In [88]:
fmin = 5.725e9
fMAX = 5.875e9
f0 = 5.8e9
B = fMAX - fmin
T = 600e-6
alpha = 2 * (fMAX - fmin) / T
c = 3e8

In [7]:
t = np.arange( 0, T, 100e-9)

In [101]:
rampsPerFile = 1500
samplesPerRamp = 6000
samplesPerUpRamp = 3000
nbFiles = 10
adc = np.zeros( nbFiles * rampsPerFile * samplesPerRamp, dtype=complex )

In [112]:
def sr( t, r ):
    
    tau = 2 * r / c
    
    y = np.zeros( t.shape, dtype=complex )
    
    # up ramp
    y[0:samplesPerUpRamp] = np.exp( -1j * 2  * np.pi * f0 * tau ) \
    * np.exp( 1j * np.pi * alpha * tau**2 ) \
    * np.exp(  -1j * 2 * np.pi * alpha * t[0:samplesPerUpRamp] * tau )
    
    # down ramp
    y[samplesPerUpRamp:2*samplesPerUpRamp] = np.exp( -1j * 2  * np.pi * f0 * tau ) \
    * np.exp( 1j * np.pi * (-alpha) * tau**2 ) \
    * np.exp(  -1j * 2 * np.pi * (-alpha) * t[samplesPerUpRamp:2*samplesPerUpRamp] * tau )
    
    return y

In [126]:
def srn( t, r, n ):
    
    tau = 2 * r / c
    
    y = np.zeros( t.shape, dtype=complex )
    
    # up ramp
    y[0:samplesPerUpRamp] = np.exp( -1j * 2  * np.pi * f0 * tau ) \
    * np.exp( 1j * np.pi * alpha * tau**2 ) \
    * np.exp(  -1j * 2 * np.pi * alpha * (t[0:samplesPerUpRamp] + n*T) * tau )
    
    # down ramp
    y[samplesPerUpRamp:2*samplesPerUpRamp] = np.exp( -1j * 2  * np.pi * f0 * tau ) \
    * np.exp( 1j * np.pi * (-alpha) * tau**2 ) \
    * np.exp(  -1j * 2 * np.pi * (-alpha) * (t[samplesPerUpRamp:2*samplesPerUpRamp] + n*T) * tau )
    
    return y

Let us build the equivalent of 10 files of data

In [113]:
a = 0.603354564402
b = a * 180 / (30 * np.pi) # see the Newton method at the bottom of the notebook
def wa( az, rg0 ):
    Rn = ( rg0**2 + az**2 )**0.5
    wa = np.sinc( b * np.arccos( rg0 / Rn ) )**2
    return wa

In [114]:
V = 40
hPlane = 430
hCorner = 90
gr_corner = 100
rg_corner = ( (hPlane - hCorner)**2 + gr_corner**2 )**0.5
az_corner = 186
noise = 0
print("rg_corner = {:.2f}, gr_corner = {:.2f}".format(rg_corner, gr_corner))

rg_corner = 354.40, gr_corner = 100.00


In [115]:
(340**2+100**2)**0.5

354.400902933387

In [116]:
Laz = nbFiles * rampsPerFile * T * V
phi = np.arctan( (Laz/2) / rg_corner )
print( "Laz = {:.0f}, phi = {:.1f}°".format( Laz, phi * 180 / np.pi ) )

Laz = 360, phi = 26.9°


In [127]:
for n in range( nbFiles * rampsPerFile):
    azn = n * T * V
    Rn = ( rg_corner**2 + (azn - az_corner)**2 )**0.5
    # azmimuth beam pattern effect + 
    adc[ n * samplesPerRamp : (n+1) * samplesPerRamp ] = srn( t, Rn, n ) * wa( azn - az_corner, rg_corner ) \
    + np.random.randn( 6000 ) * noise
    if n%1000 == 0:
        phi = np.arccos( rg_corner / Rn ) * 180 / np.pi
        print( "{}, wa = {:.1f}, phi = {:.1f}".format( n, wa( azn - az_corner, rg_corner ), phi ) )

0, wa = 0.3, phi = 27.7
1000, wa = 0.4, phi = 24.6
2000, wa = 0.5, phi = 21.3
3000, wa = 0.6, phi = 17.8
4000, wa = 0.8, phi = 14.2
5000, wa = 0.9, phi = 10.5
6000, wa = 0.9, phi = 6.8
7000, wa = 1.0, phi = 2.9
8000, wa = 1.0, phi = 1.0
9000, wa = 1.0, phi = 4.8
10000, wa = 0.9, phi = 8.7
11000, wa = 0.8, phi = 12.4
12000, wa = 0.7, phi = 16.1
13000, wa = 0.6, phi = 19.6
14000, wa = 0.5, phi = 22.9


In [128]:
adc_reshaped = adc.reshape( (nbFiles * rampsPerFile, samplesPerRamp) )

In [129]:
adc

array([-0.19722575-0.2473064j , -0.31576392-0.01875101j,
       -0.22510805+0.22222695j, ..., -0.24191734+0.27197895j,
       -0.36394840+0.00617323j, -0.25100205-0.26361813j])

In [130]:
RD = build_rd_from_data_sim( adc_reshaped, samplesPerRamp )

shape of the samples matrix = (15000, 6000)
Data are NOT shifted


In [140]:
samplesPerRamp

6000

In [131]:
ifft_RD = np.fft.ifft( RD, axis=1 )

In [132]:
plt.figure()
plt.imshow( 20 * np.log10( np.abs( ifft_RD[::10,:] ) ) )
plt.colorbar()

In [143]:
ifft_adc = np.fft.ifft( adc_reshaped, axis=1 )
print( "adc_reshaped.shape = {}, ifft_adc.shape = {}".format( adc_reshaped.shape, ifft_adc.shape ) )

adc_reshaped.shape = (15000, 6000), ifft_adc.shape = (15000, 6000)


In [144]:
plt.figure()
plt.imshow( 20 * np.log10( np.abs( ifft_adc[::10,:] ) ) )
plt.colorbar()

In [57]:
az_corner / V / T

7750.000000000001

In [58]:
plt.figure()
plt.plot( 20 * np.log10( np.abs( ifft_RD[7750,:] ) ) )
plt.plot( 20 * np.log10( np.abs( ifft_RD[25000,:] ) ) )

In [59]:
plt.figure()
plt.plot( 20 * np.log10( np.abs( ifft_RD[::10,500] ) ))
plt.plot( 20 * np.log10( np.abs( ifft_RD[::10,501] ) ))

# Save simulated data

In [133]:
firstFile = 0
lastFile = nbFiles - 1
data_dir = "/home/pleroy/DATA/SIMU"

In [134]:
coupling_RD = np.average(RD, 0)
np.save( data_dir + "/coupling_RD_files_{}_{}".format(firstFile, lastFile), coupling_RD )

In [146]:
np.save(data_dir + '/RD_files_{}_{}'.format(firstFile, lastFile), adc_reshaped[:, 0:3000]) 

In [138]:
RD[100, 0:3000].size

1500

In [139]:
RD.shape

(15000, 1500)

### Newton's method
f(x) = sin(x) / x

f'(x) = cos(x) / x - sin(x) / x^2

x(n+1) = x(n) - f(x(n)) / f'(x(n))

x(n+1) = x(n) - [f(x(n)) - y0] / f'(x(n))

In [ ]:
y0 = 0.5
def newton( xn ):
    f_xn = np.sinc( xn )
    #f_prime_xn = np.cos(xn) / xn - np.sin(xn) / xn**2
    f_prime_xn = np.cos(np.pi * xn) / xn - np.sin(np.pi*xn) / (np.pi*xn**2)
    xn_plus_1 = xn - ( f_xn - y0 ) / f_prime_xn
    return xn_plus_1

In [ ]:
x = 1
for k in range( 10 ):
    x = newton( x )
    print( x )

In [ ]:
a = 0.603354564402
b = a * 180 / (30 * np.pi)

In [ ]:
x = 30 * np.pi / 180
np.sinc( b * x )